# Using Hugging Face With Skills

In this notebook, we demonstrate using Hugging Face models for Skills using both SemanticMemory and text completions. 

SK supports downloading models from the Hugging Face that can perform the following tasks: text-generation, text2text-generation, summarization, and sentence-similarity. You can search for models by task at https://huggingface.co/models.

In [1]:
!python -m pip install semantic-kernel==0.3.15.dev0

# Note that additional dependencies are required for the Hugging Face connectors:
!python -m pip install torch==2.0.0
!python -m pip install transformers==^4.28.1
!python -m pip install sentence-transformers==^2.2.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB ? eta 0:00:000:00:0100:02
  Using cached filelock-3.13.1-py3-none-any.whl.metadata (2.8 kB)
  Using cached sympy-1.12-py3-none-any.whl (5.7 MB)
  Using cached networkx-3.2.1-py3-none-any.whl.metadata (5.2 kB)
  Using cached nvidia_cuda_nvrtc_cu11-11.7.99-2-py3-none-manylinux1_x86_64.whl (21.0 MB)
  Using cached nvidia_cuda_runtime_cu11-11.7.99-py3-none-manylinux1_x86_64.whl (849 kB)
  Using cached nvidia_cuda_cupti_cu11-11.7.101-py3-none-manylinux1_x86_64.whl (11.8 MB)
  Using cached nvidia_cudnn_cu11-8.5.0.96-2-py3-none-manylinux1_x86_64.whl (557.1 MB)
  Using cached nvidia_cublas_cu11-11.10.3.66-py3-none-manylinux1_x86_64.whl (317.1 MB)
  Using cached nvidia_cufft_cu11-10.9.0.58-py3-none-manylinux1_x86_64.whl (168.4 MB)
  Using cached nvidia_curand_cu11-10.2.10.91-py3-none-manylinux1_x86_64.whl (54.6 MB)
  Using cached nvidia_cusolver_cu11-11.4.0.1-2-py3-none-manylinux1_x86_64.whl (102.6 MB)
  Using cached nvidia_cusparse_c

In [2]:
import semantic_kernel as sk
import semantic_kernel.connectors.ai.hugging_face as sk_hf

First, we will create a kernel and add both text completion and embedding services. 

For text completion, we are choosing GPT2. This is a text-generation model. (Note: text-generation will repeat the input in the output, text2text-generation will not.)
For embeddings, we are using sentence-transformers/all-MiniLM-L6-v2. Vectors generated for this model are of length 384 (compared to a length of 1536 from OpenAI ADA).

The following step may take a few minutes when run for the first time as the models will be downloaded to your local machine.

In [5]:
kernel = sk.Kernel()

# Configure LLM service
kernel.add_text_completion_service(
    "gpt2", sk_hf.HuggingFaceTextCompletion("gpt2", task="text-generation")
)
kernel.add_text_embedding_generation_service(
    "sentence-transformers/all-MiniLM-L6-v2",
    sk_hf.HuggingFaceTextEmbedding("sentence-transformers/all-MiniLM-L6-v2"),
)
kernel.register_memory_store(memory_store=sk.memory.VolatileMemoryStore())
kernel.import_skill(sk.core_skills.TextMemorySkill())

.gitattributes: 100%|██████████| 1.18k/1.18k [00:00<00:00, 7.04MB/s]
1_Pooling/config.json: 100%|██████████| 190/190 [00:00<00:00, 602kB/s]
README.md: 100%|██████████| 10.6k/10.6k [00:00<00:00, 48.3MB/s]
config_sentence_transformers.json: 100%|██████████| 116/116 [00:00<00:00, 763kB/s]
data_config.json: 100%|██████████| 39.3k/39.3k [00:00<00:00, 13.3MB/s]
pytorch_model.bin: 100%|██████████| 90.9M/90.9M [00:07<00:00, 11.5MB/s]
sentence_bert_config.json: 100%|██████████| 53.0/53.0 [00:00<00:00, 286kB/s]
special_tokens_map.json: 100%|██████████| 112/112 [00:00<00:00, 501kB/s]
tokenizer.json: 100%|██████████| 466k/466k [00:00<00:00, 6.88MB/s]
tokenizer_config.json: 100%|██████████| 350/350 [00:00<00:00, 1.90MB/s]
train_script.py: 100%|██████████| 13.2k/13.2k [00:00<00:00, 42.4MB/s]
vocab.txt: 100%|██████████| 232k/232k [00:00<00:00, 14.3MB/s]
modules.json: 100%|██████████| 349/349 [00:00<00:00, 1.50MB/s]


{'recall': <semantic_kernel.orchestration.sk_function.SKFunction at 0x7ff2f1c77f10>,
 'save': <semantic_kernel.orchestration.sk_function.SKFunction at 0x7ff2f1c9c1d0>}

### Add Memories and Define a skill to use them

Most models available on huggingface.co are not as powerful as OpenAI GPT-3+. Your skills will likely need to be simpler to accommodate this.

In [6]:
await kernel.memory.save_information_async(
    "animal-facts", id="info1", text="Sharks are fish."
)
await kernel.memory.save_information_async(
    "animal-facts", id="info2", text="Whales are mammals."
)
await kernel.memory.save_information_async(
    "animal-facts", id="info3", text="Penguins are birds."
)
await kernel.memory.save_information_async(
    "animal-facts", id="info4", text="Dolphins are mammals."
)
await kernel.memory.save_information_async(
    "animal-facts", id="info5", text="Flies are insects."
)

# Define semantic function using SK prompt template language
my_prompt = """I know these animal facts: {{recall $query1}} {{recall $query2}} {{recall $query3}} and """

# Create the semantic function
my_function = kernel.create_semantic_function(
    my_prompt, max_tokens=45, temperature=0.5, top_p=0.5
)


Let's now see what the completion looks like! Remember, "gpt2" is nowhere near as large as ChatGPT, so expect a much simpler answer.

In [8]:
context = kernel.create_new_context()
context[sk.core_skills.TextMemorySkill.COLLECTION_PARAM] = "animal-facts"
context[sk.core_skills.TextMemorySkill.RELEVANCE_PARAM] = 0.3

context["query1"] = "animal that swims"
context["query2"] = "animal that flies"
context["query3"] = "penguins are?"
output = await kernel.run_async(my_function, input_vars=context.variables)

output = str(output).strip()


query_result1 = await kernel.memory.search_async(
    "animal-facts", context["query1"], limit=1, min_relevance_score=0.3
)
query_result2 = await kernel.memory.search_async(
    "animal-facts", context["query2"], limit=1, min_relevance_score=0.3
)
query_result3 = await kernel.memory.search_async(
    "animal-facts", context["query3"], limit=1, min_relevance_score=0.3
)

print(f"gpt2 completed prompt with: '{output}'")

/home/chris/anaconda3/envs/semantic/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/chris/anaconda3/envs/semantic/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


gpt2 completed prompt with: 'I know these animal facts: Dolphins are mammals. Flies are insects. Penguins are birds. and  I'm not even sure if they're all mammals. But I do know that the Dolphins are mammals. And I'm not even sure if they're all mammals. But I do know that the Dolphins are mammals. And'
